In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile
import glob
from PIL import Image
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("CatsVsDogs")

wandb.login(key=secret_value_0)
wandb.init(project='DogsVsCatsCNN', save_code=True)

In [ ]:
train_dir = 'train'
test_dir = 'test'
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip') as train_zip:
    train_zip.extractall('')
    
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip') as test_zip:
    test_zip.extractall('')
train_list = glob.glob(os.path.join(train_dir,'*.jpg'))
test_list = glob.glob(os.path.join(test_dir, '*.jpg'))
print(f"Train Data: {len(train_list)}")
print(f"Test Data: {len(test_list)}")

In [ ]:
labels = [path.split('/')[-1].split('.')[0] for path in train_list]

In [ ]:
random_idx = np.random.randint(1, len(train_list), size=9)
fig, axes = plt.subplots(3, 3, figsize=(16, 12))

for idx, ax in enumerate(axes.ravel()):
    img = Image.open(train_list[idx])
    ax.set_title(labels[idx])
    ax.imshow(img)

In [ ]:
train_list, valid_list = train_test_split(train_list, 
                                          test_size=0.2,
                                          stratify=labels,
                                          random_state=0)
print(f"Train Data: {len(train_list)}")
print(f"Validation Data: {len(valid_list)}")
print(f"Test Data: {len(test_list)}")

In [ ]:
train_transforms = transforms.Compose([
        transforms.Resize(128), # makes it easier for the GPU
        transforms.RandomResizedCrop(112),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()])

val_transforms = transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(112),
        transforms.ToTensor()])


test_transforms = transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(112),
        transforms.ToTensor()])

In [ ]:
class CatsDogsDataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform
        self.filelength = len(file_list)

    def __len__(self):
        return self.filelength

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        label = img_path.split("/")[-1].split(".")[0]
        label = 1 if label == "dog" else 0
        return img_transformed, label

In [ ]:
train_data = CatsDogsDataset(train_list, transform=train_transforms)
valid_data = CatsDogsDataset(valid_list, transform=test_transforms)
test_data = CatsDogsDataset(test_list, transform=test_transforms)

In [ ]:
batch_size = 32
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight, nn.init.calculate_gain('relu'))
        torch.nn.init.constant_(m.bias, 0)

In [ ]:
class AlexNet(nn.Module):
    
    def __init__(self, num_classes: int):
        super(AlexNet, self).__init__()
        
        self.feature_extr = nn.Sequential(
        #First convolutional layer
            nn.Conv2d(in_channels=3, out_channels= 96, kernel_size= 11, stride=4, padding = 2, bias = False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding = 0),
        
        #Second convolutional layer
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride= 1, padding= 2, bias = False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding = 0),
        
        #Third convolutional layer
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride= 1, padding= 1, bias = False),
            nn.ReLU(inplace=True),
            
        #Forth convolutional layer
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1, bias = False),
            nn.ReLU(inplace=True),
            
        #Fifth convolutional layer
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1, bias = False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding = 0),
        )
        
        
        self.linear = nn.Sequential(
        #First fully connected layer
            nn.Linear(in_features = 1024, out_features = 4096),
            nn.ReLU(inplace = True),
            nn.Dropout(p = 0.5),
            
        #Second
            nn.Linear(in_features = 4096, out_features = 4096),
            nn.ReLU(inplace = True),
            nn.Dropout(p = 0.5),
        
        #Third
            nn.Linear(in_features = 4096 , out_features = 2),
        )
    
    def forward(self, x) -> torch.Tensor:
        x = self.feature_extr(x)
        
        x = x.reshape(x.shape[0], -1)
        
        x = self.linear(x)
        return x

In [ ]:
model = AlexNet(2)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
    
    training_loss = 0
    total = 0
    model.train()
    for X, y in train_loader:
        X = X.to(device)
        y = y.to(device)
        optimizer.zero_grad()

        y_pred = model(X)
        loss = criterion(y_pred, y)

        loss.backward()
        optimizer.step()

        training_loss += loss.item()
        _, predicted = torch.max(y_pred, 1)
        total += int(predicted.eq(y).sum().item())

    wandb.log({
            'training_loss': training_loss,
            'training_accuracy': total / len(train_list)
    })
    
    with torch.no_grad():
        validation_loss = 0
        total = 0
        for X, y in valid_loader:
            X = X.to(device)
            y = y.to(device)

            y_pred = model(X)
            _, predicted = torch.max(y_pred, 1)
            loss = criterion(y_pred, y)
            validation_loss += loss.item()
            
            total += int(predicted.eq(y).sum().item())


        wandb.log({
                'validation_loss': validation_loss,
                'validation_accuracy': total / len(valid_list)
        })
    

In [ ]:
torch.save(model.state_dict(), 'my_model.pt')